# Verbindung zu elasticsearch

In [3]:
from elasticsearch import Elasticsearch 

API_KEY="MUZsY3lwSUJyREVzOWVqdmlFTHo6aTNoOFhnWlRTXzZqb0d1dXBEeW9FUQ=="

client=Elasticsearch("https://localhost:9200/",api_key=API_KEY ,)
client.info().body

{'name': 'es01',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'hVMplK85SQGXVAkp3s3BfQ',
 'version': {'number': '8.7.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'f229ed3f893a515d590d0f39b05f68913e2d9b53',
  'build_date': '2023-04-27T04:33:42.127815583Z',
  'build_snapshot': False,
  'lucene_version': '9.5.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [4]:
import pandas as pd

df=(
   pd.read_csv("data/air_qual.csv")  
)
print(df)

       Unique ID  Indicator ID                  Name Measure Measure Info  \
0         216498           386            Ozone (O3)    Mean          ppb   
1         216499           386            Ozone (O3)    Mean          ppb   
2         219969           386            Ozone (O3)    Mean          ppb   
3         219970           386            Ozone (O3)    Mean          ppb   
4         164876           383  Sulfur Dioxide (SO2)    Mean          ppb   
...          ...           ...                   ...     ...          ...   
16117     671118           386            Ozone (O3)    Mean          ppb   
16118     671119           386            Ozone (O3)    Mean          ppb   
16119     671120           386            Ozone (O3)    Mean          ppb   
16120     671121           386            Ozone (O3)    Mean          ppb   
16121     671122           386            Ozone (O3)    Mean          ppb   

      Geo Type Name  Geo Join ID                          Geo Place Name  \

In [5]:
import pandas as pd

df=(
   pd.read_csv("data/movies.csv")  
   .dropna()
   .sample(5000,random_state=42)
   .reset_index()

)
print(df)

      index  Release Year                    Title Origin/Ethnicity  \
0     10286          1984               Songwriter         American   
1     28335          2012    Da Thadiya (ഡാ തടിയാ)        Malayalam   
2     10742          1987                 Ironweed         American   
3     28533          2015               Kaliyachan        Malayalam   
4     24808          1971              Kati Patang        Bollywood   
...     ...           ...                      ...              ...   
4995  23671          2010               Triple Tap        Hong Kong   
4996  14874          2006          Dance Party USA         American   
4997   3015          1941      Caught in the Draft         American   
4998  15350          2007  Suffering Man's Charity         American   
4999  23728          2014         Temporary Family        Hong Kong   

                 Director                                               Cast  \
0            Alan Rudolph  Willie Nelson, Kris Kristofferson, Melin

In [6]:
mappings={
    "properties":{
            "title":{"type":"text", "analyzer":"english"},
            "ethnicity":{"type":"text", "analyzer":"standard"},
            "director":{"type":"text", "analyzer":"standard"},
            "cast":{"type":"text", "analyzer":"standard"},
            "genre":{"type":"text", "analyzer":"standard"},
            "plot":{"type":"text", "analyzer":"english"},
            "year":{"type":"integer"},
            "wiki_page":{"type":"keyword"}
            }
}
client.indices.create(index="movies" ,mappings=mappings)


BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [movies/-VRJsezTTsewvPBf3g-wtA] already exists')

In [8]:
for i,row in df.iterrows():
    doc={ "title":row["Title"],
          "ethnicity":row["Origin/Ethnicity"],
            "director":row["Director"],
            "cast":row["Cast"],
            "genre":row["Genre"],
            "plot":row["Plot"],
            "year":row["Release Year"],
            "wiki_page":row["Wiki Page"]}
    client.index(index="movies" ,id=i,document=doc)

In [9]:
from elasticsearch.helpers import bulk

bulk_data=[]
for i,row in df.iterrows():
    bulk_data.append(
        { 
            "_index": "movies",
            "_id":i,
            "_source":{
                "title":row["Title"],
                "ethnicity":row["Origin/Ethnicity"],
                "director":row["Director"],
                "cast":row["Cast"],
                "genre":row["Genre"],
                "plot":row["Plot"],
                "year":row["Release Year"],
                "wiki_page":row["Wiki Page"]}
        }
    )
bulk(client,bulk_data)

(5000, [])

In [10]:
client.indices.refresh(index="movies")
client.cat.count(index="movies")


TextApiResponse('1730240869 22:27:49 5000\n')

# Suche in python

In [ ]:
rsp=client.search(index="movies",
                  query={
                      
                      "match_all":{}
                  }
)
print(rsp)

rsp1=client.search(index="movies",
                  query={
                      
                      "match":{"year":1987}
                  }
)
print(rsp1)

In [ ]:
rsp=client.search(index="movies",
                  query={
                    "bool":{
                           "must" :{
                               "match_phrase":{
                                    "cast":"jack nicholson"
                               }
                           } ,
                           "filter":{
                               "bool": {"must_not":{ 
                                   "match_phrase":{"director":"roman polanski"}
                               }}
                           }
                    } 
                  }
)
rsp.body

In [ ]:
client.delete(index="movies",id="461")

In [ ]:
client.indices.delete(index="movies")